In [ ]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import logging
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from torch import optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from utils.paths import ROOT, DATA_DIR, RAW_DIR, PROCESSED_DIR, EQUITY_DIR
from models.unet import UNet

In [ ]:
from ingestion import read_equity, TSFinDataset
from preprocessing import to_time_bars, to_volume_bars, to_dollar_bars
from models import UNet
from frameworks.diffusion import Engine, GaussianDiffusion
from utils import plot_comparison

In [ ]:
df = read_equity("AAPL")
df["Close"].plot(figsize=(15, 7))

In [ ]:
target_features = [ "Close", "Volume"]
window_size=64
dataset = TSFinDataset(
    df,
    features=target_features,
    window_size=window_size,
    use_timestamp=False,
)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
sample_batch = next(iter(dataloader))
print(f"Batch Shape: {sample_batch['x'].shape}")

In [ ]:
sample_batch['x'].shape[2]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
num_features = sample_batch['x'].shape[2]
lr = 3e-4 # Lr means learning rate

diffusion = GaussianDiffusion(noise_steps=1000, beta_start=1e-4, beta_end=0.02, length=window_size, channels=num_features)
model = UNet(c_in=num_features, c_out=num_features, time_dim=256).to(device)
optimizer = optim.AdamW(model.parameters(), lr=lr)

In [ ]:
print(next(model.parameters()).device)

In [ ]:
trainer = Engine(
    model=model,
    diffusion=diffusion,
    dataloader=dataloader,
    optimizer=optimizer,
)

In [ ]:
trainer.fit(epochs=512)

In [ ]:
trainer.plot_loss()

In [ ]:
plot_comparison(dataloader, model, diffusion, dataset.get_scaler(), dataset)